In [0]:
%sql

CREATE OR REPLACE TEMPORARY VIEW incoming_fact_order_lifecycle as( 
  WITH watermark_table as (
    select watermark_ts from otc.gold.watermark where table_name='fact_order_lifecycle'
  ),
  order_customer as (
  SELECT
    o.order_id,
    c.customer_sk,
    o.order_ts,
    o.order_status,
    o.ingest_ts AS orders_ingest_ts,
    c.ingest_ts AS customer_ingest_ts
  FROM otc.silver.orders o
  LEFT JOIN otc.silver.customer c
    ON o.customer_id = c.customer_id
   AND c.is_current = TRUE
  WHERE o.ingest_ts > (select watermark_ts from watermark_table)
),

order_customer_items as (
  SELECT
    oc.*,
    (oi.quantity * oi.unit_price) AS line_amount,
    oi.ingest_ts AS order_items_ingest_ts
  FROM order_customer oc
  LEFT JOIN otc.silver.order_items oi
    ON oc.order_id = oi.order_id
),

order_customer_items_agg as (
  SELECT
    order_id,
    customer_sk,
    order_ts,
    order_status,
    CAST(SUM(line_amount) AS DECIMAL(12,2)) AS order_total_amount,
    MAX(orders_ingest_ts)      AS orders_ingest_ts,
    MAX(customer_ingest_ts)    AS customer_ingest_ts,
    MAX(order_items_ingest_ts) AS order_items_ingest_ts
  FROM order_customer_items
  GROUP BY order_id, customer_sk, order_ts, order_status
),

order_payments as (
  SELECT
    ocia.*,
    p.payment_status,
    p.payment_ts AS last_payment_ts,
    p.ingest_ts  AS payments_ingest_ts
  FROM order_customer_items_agg ocia
  LEFT JOIN otc.silver.payments p
    ON ocia.order_id = p.order_id
),

final_fact as (
  SELECT
    op.order_id,
    op.customer_sk,
    op.order_ts,
    op.order_status,
    op.order_total_amount,
    op.payment_status,
    op.last_payment_ts,
    s.shipment_status,
    s.shipment_ts AS last_shipment_ts,
    GREATEST(
      op.orders_ingest_ts,
      op.customer_ingest_ts,
      op.order_items_ingest_ts,
      COALESCE(op.payments_ingest_ts, TIMESTAMP '1900-01-01 00:00:00'),
      COALESCE(s.ingest_ts,           TIMESTAMP '1900-01-01 00:00:00')
    ) AS record_updated_ts
  FROM order_payments op
  LEFT JOIN otc.silver.shipments s
    ON op.order_id = s.order_id
)

SELECT
  order_id,
  customer_sk,
  order_ts,
  order_status,
  order_total_amount,
  payment_status,
  last_payment_ts,
  shipment_status,
  last_shipment_ts,
  record_updated_ts
FROM final_fact
);

In [0]:
%sql
select * from incoming_fact_order_lifecycle

In [0]:
%sql
MERGE INTO otc.gold.fact_order_lifecycle as o
USING incoming_fact_order_lifecycle as i
ON o.order_id = i.order_id
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *

In [0]:
%sql
UPDATE otc.gold.watermark
  SET watermark_ts = (select max(record_updated_ts) from otc.gold.fact_order_lifecycle)
  WHERE table_name = 'fact_order_lifecycle'

In [0]:
%sql
select * from otc.gold.fact_order_lifecycle

In [0]:
%sql
select * from otc.gold.watermark